In [1]:
from sqlalchemy import select, func, and_, or_, Integer
from sqlalchemy.orm import selectinload

from models import WorkersOrm, ResumesOrm, Workload
from schemas import ResumesDTO, ResumesRelDTO, WorkersDTO, WorkersRelDTO
from database import session_factory, sync_engine

from pydantic import BaseModel

#### Без relationship

In [2]:
with session_factory() as session:
    query = (
        select(WorkersOrm)
        .limit(2)
    )

    res = session.execute(query)
    result_orm = res.scalars().all()
    print(f"{result_orm=}")
    result_dto = [WorkersDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

2025-01-04 10:21:56,703 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-01-04 10:21:56,704 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-04 10:21:56,705 INFO sqlalchemy.engine.Engine select current_schema()
2025-01-04 10:21:56,706 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-04 10:21:56,707 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-01-04 10:21:56,707 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-04 10:21:56,711 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 10:21:56,722 INFO sqlalchemy.engine.Engine SELECT workers.id, workers.username 
FROM workers 
 LIMIT %(param_1)s::INTEGER
2025-01-04 10:21:56,723 INFO sqlalchemy.engine.Engine [generated in 0.00106s] {'param_1': 2}
result_orm=[<WorkersOrm id=1, username=Jack>, <WorkersOrm id=2, username=Michael>]
result_dto=[WorkersDTO(username='Jack', id=1), WorkersDTO(username='Michael', id=2)]
2025-01-04 10:21:56,726 INFO sqlalchemy.engine.Engine ROLLBACK


C:\Users\misto\AppData\Local\Temp\ipykernel_13348\3179573061.py:7: SAWarning: relationship 'WorkersOrm.resumes_parttime' will copy column workers.id to column resumes.worker_id, which conflicts with relationship(s): 'WorkersOrm.resumes' (copies workers.id to resumes.worker_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="resumes"' to the 'WorkersOrm.resumes_parttime' relationship. (Background on this warning at: https://sqlalche.me/e/20/qzyx) (This warning originated from the `configure_mappers()` process, which was invoked automatically in response to a user-initiated operation.)
  res = session.execute(query)


#### C relationship

In [3]:
with session_factory() as session:
    query = (
        select(WorkersOrm)
        .options(selectinload(WorkersOrm.resumes))
        .limit(2)
    )

    res = session.execute(query)
    result_orm = res.scalars().all()
    print(f"{result_orm=}")
    result_dto = [WorkersRelDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

2025-01-04 10:22:05,154 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 10:22:05,157 INFO sqlalchemy.engine.Engine SELECT workers.id, workers.username 
FROM workers 
 LIMIT %(param_1)s::INTEGER
2025-01-04 10:22:05,157 INFO sqlalchemy.engine.Engine [generated in 0.00085s] {'param_1': 2}
2025-01-04 10:22:05,161 INFO sqlalchemy.engine.Engine SELECT resumes.worker_id AS resumes_worker_id, resumes.id AS resumes_id, resumes.title AS resumes_title, resumes.compensation AS resumes_compensation, resumes.workload AS resumes_workload, resumes.created_at AS resumes_created_at, resumes.updated_at AS resumes_updated_at 
FROM resumes 
WHERE resumes.worker_id IN (%(primary_keys_1)s::INTEGER, %(primary_keys_2)s::INTEGER)
2025-01-04 10:22:05,162 INFO sqlalchemy.engine.Engine [generated in 0.00102s] {'primary_keys_1': 1, 'primary_keys_2': 2}
result_orm=[<WorkersOrm id=1, username=Jack>, <WorkersOrm id=2, username=Michael>]
result_dto=[WorkersRelDTO(username='Jack', id=1, resumes=[ResumesDTO(tit

#### JOIN

In [4]:
class WorkloadAvgCompensationDTO(BaseModel):
    workload: Workload
    avg_compensation: int

In [5]:
with session_factory() as session:
    query = (
        select(
            ResumesOrm.workload,
            func.avg(ResumesOrm.compensation).cast(Integer).label("avg_compensation"),
        )
        .select_from(ResumesOrm)
        .filter(and_(
            ResumesOrm.title.contains("Python"),
            ResumesOrm.compensation > 40000,
        ))
        .group_by(ResumesOrm.workload)
        .having(func.avg(ResumesOrm.compensation) > 70000)
    )
    res = session.execute(query)
    result_orm = res.all()
    print(f"{result_orm=}")
    result_dto = [WorkloadAvgCompensationDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

2025-01-04 10:22:11,379 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 10:22:11,382 INFO sqlalchemy.engine.Engine SELECT resumes.workload, CAST(avg(resumes.compensation) AS INTEGER) AS avg_compensation 
FROM resumes 
WHERE (resumes.title LIKE '%%' || %(title_1)s::VARCHAR || '%%') AND resumes.compensation > %(compensation_1)s::INTEGER GROUP BY resumes.workload 
HAVING avg(resumes.compensation) > %(avg_1)s::INTEGER
2025-01-04 10:22:11,382 INFO sqlalchemy.engine.Engine [generated in 0.00068s] {'title_1': 'Python', 'compensation_1': 40000, 'avg_1': 70000}
result_orm=[(<Workload.parttime: 'parttime'>, 165000), (<Workload.fulltime: 'fulltime'>, 90000)]
result_dto=[WorkloadAvgCompensationDTO(workload=<Workload.parttime: 'parttime'>, avg_compensation=165000), WorkloadAvgCompensationDTO(workload=<Workload.fulltime: 'fulltime'>, avg_compensation=90000)]
2025-01-04 10:22:11,385 INFO sqlalchemy.engine.Engine ROLLBACK
